# 数独とマインスイーパー

## ロジック推理問題

* A: 私はやっていない。
* B: Dは犯人です。
* C: Bは犯人だ。
* D: Bは嘘を言っています。

四人の中一人だけ本当のことを言っています。犯人はだれでしょうか。

|  容疑者 | 発言  | ブール式  |
|:--|:--|:--|
| A  | 私はやっていない  | `S1=~A`  |
| B  | Dは犯人です  |  `S2=D` |
| C  | Bは犯人だ  |  `S3=B` |
| D  | Bは嘘を言っています | `S4=~D`  |

一人だけ真実を語っています。AND-ORの選言標準形(DNF)で表すと、

```
     S1 & ~S2 & ~S3 & ~S4 |
    ~S1 &  S2 & ~S3 & ~S4 | 
    ~S1 & ~S2 &  S3 & ~S4 | 
    ~S1 & ~S2 & ~S3 &  S4
```

OR-ANDの連言標準形(CNF)で表すと、

```
    ~(S1 & S2) &
    ~(S1 & S3) &
    ~(S1 & S4) &
    ~(S2 & S3) &
    ~(S2 & S4) &
    ~(S3 & S4) &
    ~(~S1 & ~S2 & ~S3 & ~S4)
```

A, B, C, Dを入れ替えると次の式になります。

```
    ~S1, ~S2               A, ~D
    ~S1, ~S3               A, ~B
    ~S1, ~S4               A,  D
    ~S2, ~S3              ~D, ~B
    ~S2, ~S4              ~D,  D
    ~S3, ~S4              ~B,  D
     S1,  S2, S3, S4      ~A,  D, B, ~D
```     

C言語のライブラリ`picosat`でCNF式を解けます。PythonからC言語のライブラリを簡単に実行できるCythonというコンパイラを利用して、Pythonの拡張ライブラリを作ります。

In [2]:
from cycosat import CoSAT

sat = CoSAT()
problem = [[1, -4], [1, -2], [1, 4], [-4, -2],
           [-4, 4], [-2, 4], [-1, 4, 2, -4]]

sat.add_clauses(problem)
print(sat.solve())

[1, -1, -1, -1]


## 数独

In [7]:
from utils import display_sudoku

sudoku_str = """
000000185
007030000
000021400
800000020
003905600
050000004
004860000
000040300
931000000"""

sudoku = np.array([[int(x) for x in line]
                   for line in sudoku_str.strip().split()])

display_sudoku(sudoku)

,,,,,,1,8,5
,,7,,3,,,,
,,,,2,1,4,,
8,,,,,,,2,
,,3,9,,5,6,,
,5,,,,,,,4
,,4,8,6,,,,
,,,,4,,3,,
9,3,1,,,,,,


3×3のブロックに区切られた 9×9の正方形の枠内に1〜9までの数字を入れるペンシルパズル。https://ja.wikipedia.org/wiki/%E6%95%B0%E7%8B%AC

各個枠に九つのブール変数を振り分け、`bools[行, 列, 数字]`。例えば、`bools[1, 2, 3] == True`の場合、1行、2列の数値は3です。

* 各個枠に一つの数字を入れる

> 例えば、1行1列目のブロックに対して、`bools[1, 1, 1..9]`の九つの変数の中一つしか`True`です。

* 各個列に重複な数字がない

> 例えば、2列目には一つの1しかない： `bools[1..9, 2, 1]`の九つの変数の中一つしか`True`です。

* 各個行に重複な数字がない

> 例えば、3行目には一つの6しかない： `bools[3, 1..9, 6]`の九つの変数の中一つしか`True`です。

* 各個ブロックに重複な数字がない

> 例えば、1ブロック目には一つの7しかない： `bools[1..3, 1..3, 7]`の九つの変数の中一つしか`True`です。

* 初期数字があるブロック

> 例えば、7列1行目には1がある：`bools[1, 7, 1]`は`True`です。

In [8]:
import numpy as np
from itertools import combinations
from cycosat import CoSAT
from utils import display_sudoku

class SudokuSolver:
    def __init__(self):

        index = np.array(list(combinations(range(9), 2)))
        self.bools = bools = np.arange(1, 9*9*9+1).reshape(9, 9, 9)

        def get_conditions(bools):
            conditions = []
            conditions.extend( bools.reshape(-1, 9).tolist() ) 
            conditions.extend( (-bools[:,:,index].reshape(-1, 2)).tolist() ) 
            return conditions
            
        c1 = get_conditions(bools) 
        c2 = get_conditions( np.swapaxes(bools, 1, 2) ) 
        c3 = get_conditions( np.swapaxes(bools, 0, 2) ) 
        
        tmp = np.swapaxes(bools.reshape(3, 3, 3, 3, 9), 1, 2).reshape(9, 9, 9)
        c4 = get_conditions( np.swapaxes(tmp, 1, 2) )
        
        self.conditions = []
        for c in (c1, c2, c3, c4):
            self.conditions.extend(c)
            
    def solve(self):
        cells = self.cells
        sat = CoSAT()
        sat.add_clauses(self.conditions)
        assumes = [self.bools[r, c, v-1] for (r, c), v in cells.items()]       
        solution = sat.assume_solve(assumes)
        if isinstance(solution, list):
            res = np.array(sat.solve())
            mask = (res > 0).reshape(9, 9, 9)
            return (np.where(mask)[2]+1).reshape(9, 9)
        else:
            return None
        
    def show_solution(self):
        display_sudoku(self.solve(), highlights=self.cells.keys())
        
    def load_str(self, sudoku_str):
        sudoku = np.array([[int(x) for x in line]
                   for line in sudoku_str.strip().split()])
        rows, cols = np.where(sudoku != 0)
        vals = sudoku[rows, cols]
        self.cells = {(r, c): v for r, c, v in zip(rows, cols, vals)}

In [9]:
solver = SudokuSolver()
solver.load_str(sudoku_str)
solver.show_solution()

3,6,2,7,9,4,1,8,5
4,1,7,5,3,8,2,6,9
5,9,8,6,2,1,4,3,7
8,7,9,4,1,6,5,2,3
2,4,3,9,7,5,6,1,8
1,5,6,3,8,2,7,9,4
7,2,4,8,6,3,9,5,1
6,8,5,1,4,9,3,7,2
9,3,1,2,5,7,8,4,6


## マインスイーパー

マインスイーパーも数独と同じように、CNF式に変換できます。